In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Datos import Datos
import EstrategiaParticionado
import Clasificador

import numpy as np
import random

from sklearn.preprocessing import OneHotEncoder

# USAMOS NUESTRAS FUNCIONES

## Prueba sin particionado de datos

In [21]:
dataset_ttt = Datos("../ConjuntosDatos/tic-tac-toe.data")
ndata, nfeat = dataset_ttt.datos.shape
print(ndata)

958


In [22]:
np.random.shuffle(dataset_ttt.datos)
NB = Clasificador.ClasificadorNaiveBayes()
NB.entrenamiento(dataset_ttt.datos[:int(ndata*(7/8))], dataset_ttt.nominalAtributos, dataset_ttt.diccionario)
pred = NB.clasifica(dataset_ttt.datos[int(ndata*(7/8)):], dataset_ttt.nominalAtributos, dataset_ttt.diccionario)
print(pred)
print("\n")

ydatos = dataset_ttt.datos[int(ndata*(7/8)):,-1]

error = NB.error(ydatos, pred)
print(error)

[1 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1
 1 0 0 1 1 0 0 1 1]


0.375


 ## Validación Simple

In [23]:
strat_simple = EstrategiaParticionado.ValidacionSimple(0.8)

In [24]:
NB.validacion(strat_simple, dataset_ttt, NB)

[0.29166666666666663]

In [25]:
strat_simple_rep3 = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

In [26]:
NB.validacion(strat_simple_rep3, dataset_ttt, NB)

[0.36458333333333337, 0.265625, 0.36458333333333337]

## Validación Cruzada

In [27]:
strat_cross = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

In [28]:
NB.validacion(strat_cross, dataset_ttt, NB)

[0.3089005235602095,
 0.24083769633507857,
 0.2774869109947644,
 0.31937172774869105,
 0.2827225130890052]

### GERMAN

In [29]:
dataset_ger = Datos("../ConjuntosDatos/german.data")
ndata, nfeat = dataset_ger.datos.shape
print(ndata, nfeat)

1000 21


In [30]:
strat_simple_ger = EstrategiaParticionado.ValidacionSimple(0.8)
NB.validacion(strat_simple_ger, dataset_ger, NB)

[0.27]

In [31]:
strat_simple_rep3_ger = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)
NB.validacion(strat_simple_rep3_ger, dataset_ger, NB)

[0.28, 0.21499999999999997, 0.24]

In [32]:
strat_cross_ger = EstrategiaParticionado.ValidacionCruzada(k_fold=5)
NB.validacion(strat_cross_ger, dataset_ger, NB)

[0.22999999999999998, 0.27, 0.21999999999999997, 0.29500000000000004, 0.26]

# USAMOS SKLEARN (Sin particionados)

## Caso Discreto

In [38]:
ttt_db = Datos('../ConjuntosDatos/tic-tac-toe.data')
np.random.shuffle(ttt_db.datos)
X = ttt_db.datos[:,:-1]
y = ttt_db.datos[:,-1]



enc = OneHotEncoder(sparse=False)
X = np.array(enc.fit_transform(X)) 

strat_simple_ger_sk = EstrategiaParticionado.ValidacionSimple(0.8)

NBSK.validacion(strat_simple_ger_sk, )

ValueError: could not broadcast input array from shape (958,27) into shape (958,9)

In [36]:
NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False)

In [37]:
NBSK.validacion(strat_simple_ger_sk, X, NBSK)

AttributeError: 'numpy.ndarray' object has no attribute 'datos'

In [18]:
1 - clf.score(x_test,y_test)

0.30729166666666663

## Caso Continuo

In [19]:
dataset_ger = Datos("../ConjuntosDatos/german.data")
np.random.shuffle(dataset_ger.datos)
X = dataset_ger.datos[:,:-1]
y = dataset_ger.datos[:,-1]
n, m = X.shape
m_d = 0
for i in range (m):
    if(dataset_ger.nominalAtributos[i] is True):
        m_d += 1
m_c = m - m_d       
X_d = np.zeros((n, m_d))
X_c = np.zeros((n, m_c))

j = 0
k = 0
for i in range (m):
    if(dataset_ger.nominalAtributos[i] is True):
        X_d[:, j] = X[:, i]
        j+=1
    else:
        X_c[:, k] = X[:, i]
        k+=1
        
enc = OneHotEncoder(sparse=False)
X_d = np.array(enc.fit_transform(X_d)) 


x_train_d = X_d[:int(0.8*n), :]
y_train = y[:int(0.8*n)]

print(x_train_d.shape, y_train.shape)

x_test_d = X_d[int(0.8*n):, :]
y_test = y[int(0.8*n):]

x_train_c = X_c[:int(0.8*n), :]
x_test_c = X_c[int(0.8*n):, :]
print(x_train_c.shape, y_train.shape)
print(m_c, m_d)

(800, 54) (800,)
(800, 7) (800,)
7 13


In [20]:
clf_d = MultinomialNB()
clf_c = GaussianNB()
clf_d.fit(x_train_d,y_train)
clf_c.fit(x_train_c,y_train)
Error_d = 1 - clf_d.score(x_test_d,y_test)
Error_c = 1 - clf_c.score(x_test_c,y_test)
TotalError = (m_c*Error_c + m_d*Error_d) / m
print(TotalError)

0.25025
